# Kayak Project

## Imports

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import requests
from dotenv import load_dotenv
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess
from bs4 import BeautifulSoup
import requests
import time
import boto3
import datetime

load_dotenv()
%load_ext dotenv
%dotenv

# Openweathermap API
key = os.getenv('APIKEY')
# RDS Connection
DBHOST = os.getenv("DBHOST")
DBUSER = os.getenv("DBUSER")
DBPASS = os.getenv("DBPASS")
DBNAME = os.getenv("DBNAME")

In [2]:
# Importing the cities list
cities = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]

In [3]:
# Creating a dataframe with the cities list
df_cities = pd.DataFrame(columns=["city"])
df_cities['city'] = cities
df_cities.reset_index(inplace=True)
df_cities.rename(columns={'index': 'id'}, inplace=True)
df_cities

,id,city
0,0,Mont Saint Michel
1,1,St Malo
2,2,Bayeux
3,3,Le Havre
4,4,Rouen
5,5,Paris
6,6,Amiens
7,7,Lille
8,8,Strasbourg
9,9,Chateau du Haut Koenigsbourg


## Scraping Booking.com

In [4]:
# Creating a copy of the dataframe
df_booking = df_cities.copy(deep=True)

# Defining the spider class
class BookingSpider(scrapy.Spider):
    name = "Booking_data"
    cities = df_booking["city"]
    start_urls = ['https://www.booking.com/index.fr.html']

    def parse(self, response):
        for i in cities:
            yield scrapy.FormRequest.from_response(
            response,
            formdata={'ss': i },
            callback=self.after_search
        )
    # Important note: the classes names change often and may not be correct at a later date
    def after_search(self, response):
        cities = response.url.split("ss=")[-1].split("&")[0]
                
        booking = response.css('.d4924c9e74')
        
        for k in booking:
            
            yield {
                'location': cities,
                'name': k.css('a div.fcab3ed991.a23c043802::text').getall(),
                'url': k.css('h3.a4225678b2 a::attr(href)').getall(),
            }

        try:
            next_page = response.css('a.paging-next').attrib["href"]
        except KeyError:
            logging.info('No next page. Terminating crawling process.')
        else:
            yield response.follow(next_page, callback=self.after_search)

In [5]:
# Initializing the crawler process
filename = "cities.json"

if filename in os.listdir():
        os.remove(filename)

process = CrawlerProcess(settings = {
    'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0',
    'LOG_LEVEL': logging.INFO,
    "FEEDS": {
        filename: {"format": "json"},
    },
    "AUTOTHROTTLE_ENABLED": True
})

process.crawl(BookingSpider)
process.start()

2022-04-14 22:32:20 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: scrapybot)
2022-04-14 22:32:20 [scrapy.utils.log] INFO: Versions: lxml 4.8.0.0, libxml2 2.9.12, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.2.0, Python 3.9.11 (main, Mar 29 2022, 19:08:29) - [GCC 7.5.0], pyOpenSSL 22.0.0 (OpenSSL 1.1.1n  15 Mar 2022), cryptography 36.0.0, Platform Linux-5.13.0-39-generic-x86_64-with-glibc2.31
2022-04-14 22:32:20 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'LOG_LEVEL': 20,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) '
               'Gecko/20100101 Firefox/92.0'}
2022-04-14 22:32:20 [scrapy.extensions.telnet] INFO: Telnet Password: fae05b61e55cb43f
2022-04-14 22:32:20 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogS

In [5]:
# Reading the scraped json data into a new dataframe
df = pd.read_json("cities.json")
df.head()

,location,name,url
0,Mont+Saint+Michel,"[Hôtel Vert, Le Relais Saint Michel, La Mère P...",[https://www.booking.com/hotel/fr/vert.fr.html...
1,St+Malo,"[Hotel Eden, Hotel d'Aleth, Hotel Ajoncs d'Or,...",[https://www.booking.com/hotel/fr/eden-saint-m...
2,Bayeux,"[ibis budget Bayeux, Hôtel De Brunville et La ...",[https://www.booking.com/hotel/fr/etap-bayeux....
3,Le+Havre,"[Holiday Inn Express - Le Havre Centre, Nomad ...",[https://www.booking.com/hotel/fr/campanile-le...
4,Rouen,"[Radisson Blu Hotel, Rouen Centre, Maison hype...",[https://www.booking.com/hotel/fr/radisson-blu...


In [6]:
# Keeping only the first 20 items from each list
for i in range (len(df["location"])):
    df["name"][i] = df["name"][i][:20]
    df["url"][i] = df["url"][i][:20]


In [7]:
# Initializing the scraper with BeautifulSoup
lat_full = []
lon_full = []
description_full = []
score_full = []

navigator = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'

for i in range(len(df["url"])):
    lat_list = []
    lon_list = []
    description_list = []
    score_list = []
    hotel_list = df["url"][i]

    for j in hotel_list:

        # In some cases, the process will return an error on the first attempt
        try:
            page = requests.get(j, headers={'User-Agent': navigator})
            soup = BeautifulSoup(page.text, 'html.parser')
        except:
            page = requests.get(j, headers={'User-Agent': navigator})
            soup = BeautifulSoup(page.text, 'html.parser')

        lat_list.append(soup.select('p.address.address_clean a')[0].get("data-atlas-latlng").split(",")[0])
        lon_list.append(soup.select('p.address.address_clean a')[0].get("data-atlas-latlng").split(",")[1])
        description_list.append(soup.select('div#property_description_content')[0].get_text())

        try:
            score_list.append(soup.select('div.b5cd09854e.d10a6220b4')[0].get_text())
            
        except:
            # If the score is missing, setting the default value to 5.0
            score_list.append("5.0")

        # Setting up a wait timer to avoid spamming the website
        time.sleep(1.4)
        
    lat_full.append(lat_list)
    lon_full.append(lon_list)
    description_full.append(description_list)
    score_full.append(score_list)
    
    # Printing the progress, the whole process takes a bit more than 40 minutes
    print (f"{df['location'].iloc[i]} completed")

# Adding new columns based on lists of lists gathered above    
df["lat"] = lat_full
df["lon"] = lon_full
df["description"] = description_full
df["score"] = score_full
print('Task completed')

Mont+Saint+Michel completed
St+Malo completed
Bayeux completed
Le+Havre completed
Rouen completed
Paris completed
Amiens completed
Lille completed
Strasbourg completed
Chateau+du+Haut+Koenigsbourg completed
Colmar completed
Eguisheim completed
Besancon completed
Dijon completed
Grenoble completed
Annecy completed
La+Rochelle completed
Bayonne completed
Biarritz completed
Montauban completed
Toulouse completed
Ariege completed
Carcassonne completed
Collioure completed
Saintes+Maries+de+la+mer completed
Aigues+Mortes completed
Nimes completed
Uzes completed
Avignon completed
Aix+en+Provence completed
Marseille completed
Cassis completed
Bormes+les+Mimosas completed
Gorges+du+Verdon completed
Lyon completed
Task completed


In [8]:
# Replacing the + in cities names and reindexing the dataframe to add the id column that will be useful to merge later
df["location"] = df["location"].str.replace("+", " ", regex=True)
df = df.set_index("location")
df = df.reindex(cities)
df.reset_index(inplace=True)
df.rename(columns={'index': 'id'}, inplace=True)
df.head()

,name,url,lat,lon,description,score
location,,,,,,
Mont Saint Michel,"[Hôtel Vert, Le Relais Saint Michel, La Mère P...",[https://www.booking.com/hotel/fr/vert.fr.html...,"[48.61470049, 48.61758727, 48.63508532, 48.614...","[-1.50961697, -1.51039615, -1.51053965, -1.510...",[\nVous pouvez bénéficier d'une réduction Geni...,"[8,1, 7,8, 7,2, 8,2, 7,3, 7,2, 8,0, 8,1, 7,2, ..."
St Malo,"[Hotel Eden, Hotel d'Aleth, Hotel Ajoncs d'Or,...",[https://www.booking.com/hotel/fr/eden-saint-m...,"[48.66190919, 48.63593081, 48.64735692, 48.647...","[-1.98966533, -2.02171236, -2.02519655, -2.028...",[\nVous pouvez bénéficier d'une réduction Geni...,"[7,3, 7,9, 8,5, 7,4, 9,2, 7,9, 9,7, 8,0, 8,2, ..."
Bayeux,"[ibis budget Bayeux, Hôtel De Brunville et La ...",[https://www.booking.com/hotel/fr/etap-bayeux....,"[49.25424209, 49.27815769, 49.26974024, 49.277...","[-0.64648747, -0.70351392, -0.68883955, -0.701...",[\nL’ibis budget Bayeux vous accueille entre C...,"[8,2, 7,9, 8,1, 9,8, 9,4, 8,7, 8,6, 8,6, 8,7, ..."
Le Havre,"[Holiday Inn Express - Le Havre Centre, Nomad ...",[https://www.booking.com/hotel/fr/campanile-le...,"[49.49823800, 49.49331760, 49.49641693, 49.490...","[0.12883700, 0.12530688, 0.15151234, 0.0977469...","[\nL’Holiday Inn Express - Le Havre Centre, si...","[8,2, 8,6, 7,5, 8,7, 8,5, 7,2, 8,5, 8,0, 8,4, ..."
Rouen,"[Radisson Blu Hotel, Rouen Centre, Maison hype...",[https://www.booking.com/hotel/fr/radisson-blu...,"[49.44644100, 49.44465180, 49.44306423, 49.443...","[1.09412000, 1.09132620, 1.08660311, 1.0945739...",[\nVous pouvez bénéficier d'une réduction Geni...,"[8,9, 8,8, 8,6, 8,2, 7,1, 8,2, 8,2, 9,1, 7,9, ..."


In [9]:
df.reset_index(inplace=True)
df.rename(columns={'index': 'id'}, inplace=True)
df.head()

,location,name,url,lat,lon,description,score
0,Mont Saint Michel,"[Hôtel Vert, Le Relais Saint Michel, La Mère P...",[https://www.booking.com/hotel/fr/vert.fr.html...,"[48.61470049, 48.61758727, 48.63508532, 48.614...","[-1.50961697, -1.51039615, -1.51053965, -1.510...",[\nVous pouvez bénéficier d'une réduction Geni...,"[8,1, 7,8, 7,2, 8,2, 7,3, 7,2, 8,0, 8,1, 7,2, ..."
1,St Malo,"[Hotel Eden, Hotel d'Aleth, Hotel Ajoncs d'Or,...",[https://www.booking.com/hotel/fr/eden-saint-m...,"[48.66190919, 48.63593081, 48.64735692, 48.647...","[-1.98966533, -2.02171236, -2.02519655, -2.028...",[\nVous pouvez bénéficier d'une réduction Geni...,"[7,3, 7,9, 8,5, 7,4, 9,2, 7,9, 9,7, 8,0, 8,2, ..."
2,Bayeux,"[ibis budget Bayeux, Hôtel De Brunville et La ...",[https://www.booking.com/hotel/fr/etap-bayeux....,"[49.25424209, 49.27815769, 49.26974024, 49.277...","[-0.64648747, -0.70351392, -0.68883955, -0.701...",[\nL’ibis budget Bayeux vous accueille entre C...,"[8,2, 7,9, 8,1, 9,8, 9,4, 8,7, 8,6, 8,6, 8,7, ..."
3,Le Havre,"[Holiday Inn Express - Le Havre Centre, Nomad ...",[https://www.booking.com/hotel/fr/campanile-le...,"[49.49823800, 49.49331760, 49.49641693, 49.490...","[0.12883700, 0.12530688, 0.15151234, 0.0977469...","[\nL’Holiday Inn Express - Le Havre Centre, si...","[8,2, 8,6, 7,5, 8,7, 8,5, 7,2, 8,5, 8,0, 8,4, ..."
4,Rouen,"[Radisson Blu Hotel, Rouen Centre, Maison hype...",[https://www.booking.com/hotel/fr/radisson-blu...,"[49.44644100, 49.44465180, 49.44306423, 49.443...","[1.09412000, 1.09132620, 1.08660311, 1.0945739...",[\nVous pouvez bénéficier d'une réduction Geni...,"[8,9, 8,8, 8,6, 8,2, 7,1, 8,2, 8,2, 9,1, 7,9, ..."


## Getting API Data

### GPS Coordinates

In [10]:
# Creating another copy of the cities dataframe to store the coordinates data from Nominatim API
df_gps = df_cities.copy(deep=True)
lat_list = []
lon_list = []

for i in cities:
    # There is no Gorges du Verdon city, replacing by the closest city instead
    if i == "Gorges du Verdon":
        i = "La%20Palud-sur-Verdon"
        r = requests.get(f"https://nominatim.openstreetmap.org/search?city={i}&format=json").json()
        lat_list.append(r[0]['lat'])
        lon_list.append(r[0]['lon'])
    # Ariege is not a city, using the county instead here
    elif i == 'Ariege':
        r = requests.get(f"https://nominatim.openstreetmap.org/search?county={i}&format=json").json()
        lat_list.append(r[0]['lat'])
        lon_list.append(r[0]['lon'])
    # Replacing the whitespace by a web friendly notation
    else:
        name = i.replace(" ", "%20")
        r = requests.get(f"https://nominatim.openstreetmap.org/search?city={name}&format=json").json()
        lat_list.append(r[0]['lat'])
        lon_list.append(r[0]['lon'])

# Adding the coordinates to the dataframe   
df_gps['lat'] = lat_list
df_gps['lon'] = lon_list 

In [11]:
df_gps.head()

,id,city,lat,lon
0,0,Mont Saint Michel,48.6359541,-1.511459954959514
1,1,St Malo,48.649518,-2.0260409
2,2,Bayeux,49.2764624,-0.7024738
3,3,Le Havre,49.4938975,0.1079732
4,4,Rouen,49.4404591,1.0939658


### Weather Data

In [12]:
# Getting the weather data based on coordinates from the Openweathermap API
df_full = df_gps.copy(deep=True)
now = datetime.datetime.now()
temps_list = []
rain_pop = []
humidity_list = []
# The days list will be used to store data as items of a list for day +1 to day +7
days = list(range(1,8))

for i in df_full.itertuples():
    lat = i.lat
    lon = i.lon
    r = requests.get(f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&units=metric&appid={key}").json()
    weather_7_days = r['daily'][1:] # Getting the weather data for the next 7 days, first item is the current weather, which we don't want here
    temps = [j['feels_like']['day'] for j in weather_7_days]
    rain = [int(j['pop'] * 100) for j in weather_7_days]
    humidity = [j['humidity'] for j in weather_7_days]
    temps_list.append(temps)
    rain_pop.append(rain)
    humidity_list.append(humidity)
    
df_full['day_plus'] = [days for _ in range(len(df_full))]
df_full['felt_temperature'] = temps_list
df_full['rain_chances'] = rain_pop
df_full['humidity'] = humidity_list
# Calculation of a weather quality indicator, using absolute value of 35 - temperature (with a weight of 2), chances of rain as a percentage, and humidity as a percentage (with a weight of 0.5) - lower is better
df_full['score_weather'] = df_full.apply(lambda x: abs((35 - np.mean(x['felt_temperature'])) * 2) + np.mean(x['rain_chances']) + (np.mean(x['humidity']) / 2), axis=1) 
df_full['updated_at'] = now

In [13]:
df_full.head()

,id,city,lat,lon,day_plus,felt_temperature,rain_chances,humidity,score_weather,updated_at
0,0,Mont Saint Michel,48.6359541,-1.511459954959514,"[1, 2, 3, 4, 5, 6, 7]","[14.16, 13.55, 16.89, 18.08, 13.96, 14.37, 16.38]","[0, 0, 0, 0, 0, 0, 0]","[58, 54, 56, 54, 51, 53, 59]",66.817143,2022-04-25 11:23:08.325826
1,1,St Malo,48.649518,-2.0260409,"[1, 2, 3, 4, 5, 6, 7]","[12.72, 11.96, 14.73, 14.86, 12.56, 12.06, 13.23]","[0, 0, 0, 0, 0, 0, 0]","[67, 61, 67, 71, 62, 61, 68]",76.322857,2022-04-25 11:23:08.325826
2,2,Bayeux,49.2764624,-0.7024738,"[1, 2, 3, 4, 5, 6, 7]","[11.42, 10.69, 14.03, 13.81, 11.12, 11.53, 14.28]","[20, 0, 0, 0, 0, 0, 0]","[66, 63, 70, 73, 65, 66, 69]",81.748571,2022-04-25 11:23:08.325826
3,3,Le Havre,49.4938975,0.1079732,"[1, 2, 3, 4, 5, 6, 7]","[11.3, 10.97, 15.09, 14.99, 11.67, 12.08, 12.99]","[6, 0, 0, 0, 0, 0, 0]","[61, 59, 59, 66, 60, 64, 67]",76.545714,2022-04-25 11:23:08.325826
4,4,Rouen,49.4404591,1.0939658,"[1, 2, 3, 4, 5, 6, 7]","[12.43, 12.6, 16.87, 17.56, 13.58, 14.34, 15.9]","[5, 0, 0, 0, 0, 0, 0]","[56, 59, 53, 55, 51, 56, 51]",68.420000,2022-04-25 11:23:08.325826


## Merging Data

In [15]:
# Renaming coordinates columns to avoid confusion between city and hotel coordinates, and resetting index to merge on the id column
df.rename(columns={'lat': 'lat_hotels', 'lon': 'lon_hotels'}, inplace=True)
df.reset_index(inplace=True)
df.rename(columns={'index': 'id'}, inplace=True)
df.head()

,id,location,name,url,lat_hotels,lon_hotels,description,score
0,0,Mont Saint Michel,"[Hôtel Vert, Le Relais Saint Michel, La Mère P...",[https://www.booking.com/hotel/fr/vert.fr.html...,"[48.61470049, 48.61758727, 48.63508532, 48.614...","[-1.50961697, -1.51039615, -1.51053965, -1.510...",[\nVous pouvez bénéficier d'une réduction Geni...,"[8,1, 7,8, 7,2, 8,2, 7,3, 7,2, 8,0, 8,1, 7,2, ..."
1,1,St Malo,"[Hotel Eden, Hotel d'Aleth, Hotel Ajoncs d'Or,...",[https://www.booking.com/hotel/fr/eden-saint-m...,"[48.66190919, 48.63593081, 48.64735692, 48.647...","[-1.98966533, -2.02171236, -2.02519655, -2.028...",[\nVous pouvez bénéficier d'une réduction Geni...,"[7,3, 7,9, 8,5, 7,4, 9,2, 7,9, 9,7, 8,0, 8,2, ..."
2,2,Bayeux,"[ibis budget Bayeux, Hôtel De Brunville et La ...",[https://www.booking.com/hotel/fr/etap-bayeux....,"[49.25424209, 49.27815769, 49.26974024, 49.277...","[-0.64648747, -0.70351392, -0.68883955, -0.701...",[\nL’ibis budget Bayeux vous accueille entre C...,"[8,2, 7,9, 8,1, 9,8, 9,4, 8,7, 8,6, 8,6, 8,7, ..."
3,3,Le Havre,"[Holiday Inn Express - Le Havre Centre, Nomad ...",[https://www.booking.com/hotel/fr/campanile-le...,"[49.49823800, 49.49331760, 49.49641693, 49.490...","[0.12883700, 0.12530688, 0.15151234, 0.0977469...","[\nL’Holiday Inn Express - Le Havre Centre, si...","[8,2, 8,6, 7,5, 8,7, 8,5, 7,2, 8,5, 8,0, 8,4, ..."
4,4,Rouen,"[Radisson Blu Hotel, Rouen Centre, Maison hype...",[https://www.booking.com/hotel/fr/radisson-blu...,"[49.44644100, 49.44465180, 49.44306423, 49.443...","[1.09412000, 1.09132620, 1.08660311, 1.0945739...",[\nVous pouvez bénéficier d'une réduction Geni...,"[8,9, 8,8, 8,6, 8,2, 7,1, 8,2, 8,2, 9,1, 7,9, ..."


In [16]:
# Merging the hotels and weather dataframes, and dropping the unnecessary location column
df_complete = pd.merge(df_full, df, on='id')
df_complete.drop(columns=['location'], inplace=True)
now = datetime.datetime.now()
df_complete['updated_at'] = now
print(df_complete.shape)

# Saving the merged dataset as a csv file
df_complete.to_csv('df_complete.csv', index=False, header=True)

(35, 16)


## Storing the dataset in a S3 data lake

In [17]:
# Loading the S3 dession and resource
session = boto3.Session()
s3 = boto3.resource('s3')

In [18]:
# Uploading the file to S3
s3.Bucket('kayak-project-garp').upload_file('df_complete.csv', 'df_complete.csv')

In [19]:
# Downloading the file from S3 and checking it kept the same shape.
df_s3 = pd.read_csv('s3://kayak-project-garp/df_complete.csv')
df_s3.shape

(35, 16)

## Transforming

In [20]:
# Splitting the dataframe into 2 tables

df_weather = df_s3[['id', 'city', 'lat', 'lon', 'day_plus', 'felt_temperature', 'rain_chances', 'humidity', 'score_weather', 'updated_at']]
df_hotels = df_s3[['id', 'city', 'name', 'url', 'lat_hotels', 'lon_hotels', 'description', 'score', 'score_weather']]

In [21]:
df_weather.head()

,id,city,lat,lon,day_plus,felt_temperature,rain_chances,humidity,score_weather,updated_at
0,0,Mont Saint Michel,48.635954,-1.511460,"[1, 2, 3, 4, 5, 6, 7]","[14.16, 13.55, 16.89, 18.08, 13.96, 14.37, 16.38]","[0, 0, 0, 0, 0, 0, 0]","[58, 54, 56, 54, 51, 53, 59]",66.817143,2022-04-25 11:25:51.588505
1,1,St Malo,48.649518,-2.026041,"[1, 2, 3, 4, 5, 6, 7]","[12.72, 11.96, 14.73, 14.86, 12.56, 12.06, 13.23]","[0, 0, 0, 0, 0, 0, 0]","[67, 61, 67, 71, 62, 61, 68]",76.322857,2022-04-25 11:25:51.588505
2,2,Bayeux,49.276462,-0.702474,"[1, 2, 3, 4, 5, 6, 7]","[11.42, 10.69, 14.03, 13.81, 11.12, 11.53, 14.28]","[20, 0, 0, 0, 0, 0, 0]","[66, 63, 70, 73, 65, 66, 69]",81.748571,2022-04-25 11:25:51.588505
3,3,Le Havre,49.493898,0.107973,"[1, 2, 3, 4, 5, 6, 7]","[11.3, 10.97, 15.09, 14.99, 11.67, 12.08, 12.99]","[6, 0, 0, 0, 0, 0, 0]","[61, 59, 59, 66, 60, 64, 67]",76.545714,2022-04-25 11:25:51.588505
4,4,Rouen,49.440459,1.093966,"[1, 2, 3, 4, 5, 6, 7]","[12.43, 12.6, 16.87, 17.56, 13.58, 14.34, 15.9]","[5, 0, 0, 0, 0, 0, 0]","[56, 59, 53, 55, 51, 56, 51]",68.420000,2022-04-25 11:25:51.588505


In [22]:
df_hotels.head()

,id,city,name,url,lat_hotels,lon_hotels,description,score,score_weather
0,0,Mont Saint Michel,"['Hôtel Vert', 'Le Relais Saint Michel', 'La M...",['https://www.booking.com/hotel/fr/vert.fr.htm...,"['48.61470049', '48.61758727', '48.63508532', ...","['-1.50961697', '-1.51039615', '-1.51053965', ...","[""\nVous pouvez bénéficier d'une réduction Gen...","['8,1', '7,8', '7,2', '8,2', '7,3', '7,2', '8,...",66.817143
1,1,St Malo,"['Hotel Eden', ""Hotel d'Aleth"", ""Hotel Ajoncs ...",['https://www.booking.com/hotel/fr/eden-saint-...,"['48.66190919', '48.63593081', '48.64735692', ...","['-1.98966533', '-2.02171236', '-2.02519655', ...","[""\nVous pouvez bénéficier d'une réduction Gen...","['7,3', '7,9', '8,5', '7,4', '9,2', '7,9', '9,...",76.322857
2,2,Bayeux,"['ibis budget Bayeux', 'Hôtel De Brunville et ...",['https://www.booking.com/hotel/fr/etap-bayeux...,"['49.25424209', '49.27815769', '49.26974024', ...","['-0.64648747', '-0.70351392', '-0.68883955', ...",['\nL’ibis budget Bayeux vous accueille entre ...,"['8,2', '7,9', '8,1', '9,8', '9,4', '8,7', '8,...",81.748571
3,3,Le Havre,"['Holiday Inn Express - Le Havre Centre', 'Nom...",['https://www.booking.com/hotel/fr/campanile-l...,"['49.49823800', '49.49331760', '49.49641693', ...","['0.12883700', '0.12530688', '0.15151234', '0....","[""\nL’Holiday Inn Express - Le Havre Centre, s...","['8,2', '8,6', '7,5', '8,7', '8,5', '7,2', '8,...",76.545714
4,4,Rouen,"['Radisson Blu Hotel, Rouen Centre', 'Maison h...",['https://www.booking.com/hotel/fr/radisson-bl...,"['49.44644100', '49.44465180', '49.44306423', ...","['1.09412000', '1.09132620', '1.08660311', '1....","[""\nVous pouvez bénéficier d'une réduction Gen...","['8,9', '8,8', '8,6', '8,2', '7,1', '8,2', '8,...",68.420000


In [23]:
# The csv has converted the lists into strings, we need to revert that first for both dataframes
df_weather_final = pd.DataFrame()
df_weather_final[['id', 'city', 'lat', 'lon', 'score_weather']] = df_weather[['id', 'city', 'lat', 'lon', 'score_weather']]
df_weather_final['day_plus'] = df_weather['day_plus'].apply(eval)
df_weather_final['felt_temperature'] = df_weather['felt_temperature'].apply(eval)
df_weather_final['rain_chances'] = df_weather['rain_chances'].apply(eval)
df_weather_final['humidity'] = df_weather['humidity'].apply(eval)

df_hotels_final = pd.DataFrame()
df_hotels_final[['id', 'city', 'score_weather']] = df_hotels[['id', 'city', 'score_weather']]
df_hotels_final['name'] = df_hotels['name'].apply(eval)
df_hotels_final['url'] = df_hotels['url'].apply(eval)
df_hotels_final['lat_hotels'] = df_hotels['lat_hotels'].apply(eval)
df_hotels_final['lon_hotels'] = df_hotels['lon_hotels'].apply(eval)
df_hotels_final['description'] = df_hotels['description'].apply(eval)
df_hotels_final['score'] = df_hotels['score'].apply(eval)

# We need to convert coordinates to numeric type as well
df_weather_final[['lat', 'lon']] = df_weather[['lat', 'lon']].apply(pd.to_numeric)

In [24]:
# We need to sort the data by weather score and explode the lists to prepare the data for warehousing and plotting
df_weather_full = df_weather_final.sort_values('score_weather')
df_weather_full.reset_index(inplace=True, drop=True)
df_weather_full = df_weather_full.apply(pd.Series.explode)
df_weather_full[['day_plus', 'felt_temperature', 'rain_chances', 'humidity']] = df_weather_full[['day_plus', 'felt_temperature', 'rain_chances', 'humidity']].apply(pd.to_numeric)

df_hotels_full = df_hotels_final.sort_values('score_weather')
df_hotels_full.reset_index(inplace=True, drop=True)
df_hotels_full = df_hotels_full.apply(pd.Series.explode)
df_hotels_full['description'] = df_hotels_full['description'].replace("\\n", "", regex=True)
df_hotels_full['score'] = df_hotels_full['score'].replace(",", ".", regex=True)
df_hotels_full[['lat_hotels', 'lon_hotels', 'score']] = df_hotels_full[['lat_hotels', 'lon_hotels', 'score']].apply(pd.to_numeric)

In [26]:
# We reset the index before pushing to SQL in order to give each line a unique id
df_weather_full_sql = df_weather_full.reset_index()
df_weather_full_sql.reset_index(inplace=True)
df_weather_full_sql.drop(columns = ['index'], inplace=True)
df_weather_full_sql.rename({'level_0': 'index'}, axis=1, inplace=True)

df_hotels_full_sql = df_hotels_full.reset_index()
df_hotels_full_sql.reset_index(inplace=True)
df_hotels_full_sql.drop(columns = ['index'], inplace=True)
df_hotels_full_sql.rename({'level_0': 'index'}, axis=1, inplace=True)

## Storing the transformed data in a RDS data warehouse

In [27]:
# Importing libraries for the data warehousing and initialization of the SQL engine
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

engine = create_engine(f"postgresql+psycopg2://{DBUSER}:{DBPASS}@{DBHOST}/{DBNAME}", echo=True)
Session = sessionmaker(bind=engine)
session = Session()

In [28]:
# Storing the two dataframes as SQL tables
df_weather_full_sql.to_sql('weather', con=engine, if_exists='replace', index=False)
df_hotels_full_sql.to_sql('hotels', con=engine, if_exists='replace', index=False)

2022-04-25 11:28:34,787 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-04-25 11:28:34,787 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-25 11:28:34,819 INFO sqlalchemy.engine.Engine select current_schema()
2022-04-25 11:28:34,819 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-25 11:28:34,845 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-04-25 11:28:34,846 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-04-25 11:28:34,875 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-04-25 11:28:34,875 INFO sqlalchemy.engine.Engine [generated in 0.00053s] {'name': 'weather'}
2022-04-25 11:28:34,912 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-04-25 11:28:34,913 INFO sqlalchemy.engine.Engine [cached sinc

700

In [29]:
# Testing the data with a query
query = text("SELECT * FROM weather LIMIT 21")
pd.read_sql(query, engine)

2022-04-25 11:28:38,272 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-04-25 11:28:38,274 INFO sqlalchemy.engine.Engine [cached since 3.399s ago] {'name': 'SELECT * FROM weather LIMIT 21'}
2022-04-25 11:28:38,310 INFO sqlalchemy.engine.Engine SELECT * FROM weather LIMIT 21
2022-04-25 11:28:38,311 INFO sqlalchemy.engine.Engine [generated in 0.00056s] {}


,index,id,city,lat,lon,score_weather,day_plus,felt_temperature,rain_chances,humidity
0,0,5,Paris,48.858890,2.320041,58.260000,1,11.47,0,55
1,1,5,Paris,48.858890,2.320041,58.260000,2,14.54,0,45
2,2,5,Paris,48.858890,2.320041,58.260000,3,18.46,0,39
3,3,5,Paris,48.858890,2.320041,58.260000,4,19.04,0,35
4,4,5,Paris,48.858890,2.320041,58.260000,5,15.89,0,40
5,5,5,Paris,48.858890,2.320041,58.260000,6,16.27,0,35
6,6,5,Paris,48.858890,2.320041,58.260000,7,16.67,0,36
7,7,7,Lille,50.636565,3.063528,66.562857,1,12.00,17,47
8,8,7,Lille,50.636565,3.063528,66.562857,2,12.17,0,50
9,9,7,Lille,50.636565,3.063528,66.562857,3,17.16,0,41


In [30]:
query = text("SELECT * FROM hotels LIMIT 40")
pd.read_sql(query, engine)

2022-04-25 11:28:41,969 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-04-25 11:28:41,970 INFO sqlalchemy.engine.Engine [cached since 7.095s ago] {'name': 'SELECT * FROM hotels LIMIT 40'}
2022-04-25 11:28:42,002 INFO sqlalchemy.engine.Engine SELECT * FROM hotels LIMIT 40
2022-04-25 11:28:42,003 INFO sqlalchemy.engine.Engine [generated in 0.00057s] {}


,index,id,city,score_weather,name,url,lat_hotels,lon_hotels,description,score
0,0,5,Paris,58.260000,Hôtel de Banville,https://www.booking.com/hotel/fr/hoteldebanvil...,48.886394,2.294946,Vous pouvez bénéficier d'une réduction Genius ...,8.4
1,1,5,Paris,58.260000,Régence Etoile,https://www.booking.com/hotel/fr/la-regence-et...,48.876525,2.292933,Vous pouvez bénéficier d'une réduction Genius ...,8.5
2,2,5,Paris,58.260000,Room Mate Alain - Champs-Elysées,https://www.booking.com/hotel/fr/d-argentine.f...,48.874345,2.289776,Vous pouvez bénéficier d'une réduction Genius ...,8.5
3,3,5,Paris,58.260000,Westside Arc de Triomphe Hotel,https://www.booking.com/hotel/fr/westside-arc-...,48.881299,2.294803,"Doté d’un bar-salon, le Westside Arc de Triomp...",8.0
4,34,7,Lille,66.562857,Best Hôtel Lille,https://www.booking.com/hotel/fr/besthotel_lil...,50.626040,3.051903,Le Best Hôtel Lille est un hôtel-restaurant si...,7.7
5,4,5,Paris,58.260000,Hotel de L'Esperance,https://www.booking.com/hotel/fr/de-l-39-esper...,48.838158,2.349616,Vous pouvez bénéficier d'une réduction Genius ...,8.6
6,5,5,Paris,58.260000,Hotel Korner Eiffel,https://www.booking.com/hotel/fr/idealhotel.fr...,48.846590,2.288570,L’Hotel Korner Eiffel se trouve à 20 minutes d...,8.6
7,6,5,Paris,58.260000,Hôtel Aiglon,https://www.booking.com/hotel/fr/aiglon.fr.htm...,48.839712,2.330173,Vous pouvez bénéficier d'une réduction Genius ...,8.7
8,7,5,Paris,58.260000,Le Grey Hotel,https://www.booking.com/hotel/fr/grey.fr.html?...,48.881330,2.327415,Vous pouvez bénéficier d'une réduction Genius ...,8.9
9,13,5,Paris,58.260000,Hotel Petit Lafayette,https://www.booking.com/hotel/fr/holiday-villa...,48.876067,2.345675,Vous pouvez bénéficier d'une réduction Genius ...,8.8


## Plotting Maps

In [31]:
# Creating a new column to be used to determine the size of the points in the plot
df_weather_full['inverted_score'] = max(df_weather_full['score_weather']) - df_weather_full['score_weather']

### Weather Map

In [32]:
# Creating a plotly scatter mapbox showing the 5 cities with the lowest weather score
px.set_mapbox_access_token(open(".mapbox_token").read())

fig = px.scatter_mapbox(
    df_weather_full[:35],
    lat='lat',
    lon='lon',
    color='felt_temperature',
    size='inverted_score',
    color_continuous_scale=px.colors.sequential.Bluered,
    size_max=35,
    zoom=4.7,
    range_color = [min(df_weather_full[:35]['felt_temperature']), max(df_weather_full[:35]['felt_temperature'])],
    hover_name='city',
    hover_data={
        'lat': False,
        'lon': False,
        'day_plus': False,
        'rain_chances': True,
        'humidity': True,
        'felt_temperature': True,
        'inverted_score': False,
        },
    animation_frame='day_plus',
    labels={'felt_temperature': 'Felt Temperature', 'rain_chances': 'Rain Chances', 'humidity': 'Humidity'}
)

fig.update_layout(
    width = 1100,
    height = 800,
    template='plotly_dark',
    title_x=0.5,
    title_text='The 5 cities with the best weather over the next 7 days')
fig.show()

### Hotels Map

In [33]:
# Creating a plotly scatter mapbox showing the 20 hotels from the previous 5 top cities
px.set_mapbox_access_token(open(".mapbox_token").read())

fig = px.scatter_mapbox(
    df_hotels_full[:100],
    lat='lat_hotels',
    lon='lon_hotels',
    color='score',
    color_continuous_scale=px.colors.diverging.RdYlGn,
    size='score',
    size_max=25,
    zoom=4.7,
    range_color = [min(df_hotels_full[:100]['score']), max(df_hotels_full[:100]['score'])],
    hover_name='city',
    hover_data={
        'lat_hotels': False,
        'lon_hotels': False,
        'name': True,
        'url': False,
        'description': False,
        'score_weather': False,
        'id': False,
        },
    labels={'score': 'Score', 'city': 'City'}
)

fig.update_layout(
    width = 1100,
    height = 800,
    template='plotly_dark',
    title_x=0.5,
    title_text='20 best hotels from the 5 cities with the best weather')
fig.show()